In [1]:
# Library imports
import numpy as np
import pandas as pd

In [2]:
# Create Pandas Dataframe from .csv file
df = pd.read_csv("Original_File.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2752 entries, 0 to 2751
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_reference                2752 non-null   int64  
 1   order_status                   2752 non-null   object 
 2   type                           2752 non-null   object 
 3   parent_item_sku                740 non-null    object 
 4   status                         2752 non-null   object 
 5   sku                            2752 non-null   object 
 6   name                           2752 non-null   object 
 7   name_localized                 2012 non-null   object 
 8   unit_price                     2752 non-null   float64
 9   unit_cost                      2752 non-null   float64
 10  quantity                       2752 non-null   int64  
 11  total_price                    2752 non-null   int64  
 12  total_cost                     2752 non-null   i

In [4]:
df.head()

,order_reference,order_status,type,parent_item_sku,status,sku,name,name_localized,unit_price,unit_cost,quantity,total_price,total_cost,discount_amount,discount_name,tax_exclusive_unit_price,tax_exclusive_total_price,tax_exclusive_discount_amount,total_taxes,business_date
0,55221,Void,Product,NaN,Void,sk-0895,Yoga Class STUDENT - per session,درس يوغا للطلاب - لكل جلسة,25.0,0.0,1,25,0,0,-,21.73913,21.73913,0,0.00000,2023-10-01
1,55221,Void,Product,NaN,Void,sk-0903,Playland Fee,رسوم بلاي لاند,10.0,0.0,1,10,0,0,-,8.69565,8.69565,0,0.00000,2023-10-01
2,55221,Void,Product,NaN,Void,sk-0901,Climbing,التسلق,15.0,0.0,1,15,0,0,-,13.04348,13.04348,0,0.00000,2023-10-01
3,55222,Done,Product,NaN,Done,sk-0865,Yoga Class ADULT - per session,درس يوغا للكبار - لكل جلسة,45.0,0.0,1,45,0,0,-,39.13043,39.13043,0,5.86957,2023-10-01
4,55223,Void,Product,NaN,Void,sk-0870,Group Exercise & Yoga Class ADULT - per session,تمارين جماعية ودرس يوغا للكبار - لكل جلسة,45.0,0.0,1,45,0,0,-,39.13043,39.13043,0,0.00000,2023-10-01


In [5]:
# Create df copy
df_needed_cols = df.copy()

# Drop unneeded columns, based on client instructions
df_needed_cols.drop(
    ['tax_exclusive_unit_price', 
     'tax_exclusive_total_price',
     'tax_exclusive_discount_amount', 
     'total_taxes', 'discount_amount', 
     'discount_name', 'discount_amount',
     'unit_cost', 'total_cost', 
     'name_localized'], axis=1, inplace=True
)

In [6]:
# Create list of conditions for Numpy "select" method for entries based on the 1rst Modifier Option
conditions_for_df_needed = [
    (df_needed_cols["type"] == 'Product') & 
    (df_needed_cols["type"].shift(-1) == 'Modifier Option'),
    (df_needed_cols["type"].shift(-1) == 'Modifier Option') & 
    (df_needed_cols["type"].shift(-2) == 'Modifier Option')
]


# Create list of replacing values for Numpy "select" method for each column
choices_for_df_activity = [df_needed_cols["name"].shift(-1), "N/A"]
choices_for_df_parent_item_sku_1 = [df_needed_cols["parent_item_sku"].shift(-1), "N/A"]
choices_for_df_modifier_1_sku = [df_needed_cols["sku"].shift(-1), "N/A"]
choices_for_df_needed_QTI_mod_1 = [df_needed_cols["quantity"].shift(-1), "N/A"]

In [7]:
# Use Numpy.select() method with lists to create the required column for Activities
df_needed_cols["Modifier_1_Activity"] = np.select(conditions_for_df_needed, choices_for_df_activity, default="N/A")

# Use Numpy.where() method to create the required column for Trainer's Names, 
# Added in original job, column will be dropped for confidentiality purposes in GitHub file
df_needed_cols["Modifier_2_Trainer_Name"] = np.where(
    (df_needed_cols["type"].shift(-1) == 'Modifier Option') & (df_needed_cols["type"].shift(-2) == 'Modifier Option'),
    df_needed_cols["name"].shift(-2), "N/A"
)

# Use Numpy.select() method with lists to create the required column for parent sku of 1rst Modifier Option
df_needed_cols["Modifier_Parent_Item_sku"] = np.select(
    conditions_for_df_needed, 
    choices_for_df_parent_item_sku_1, 
    default="N/A"
)

# Use Numpy.where() method to create the required column for parent sku of 2nd Modifier Option
df_needed_cols["Modifier_Parent_Item_sku_2"] = np.where(
    (df_needed_cols["type"].shift(-1) == 'Modifier Option') & (df_needed_cols["type"].shift(-2) == 'Modifier Option'),
    df_needed_cols["parent_item_sku"].shift(-2), "N/A"
)

# Use Numpy.select() method with lists to create the required column for sku of 1rst Modifier Option
df_needed_cols["Modifier_1_sku"] = np.select(conditions_for_df_needed, choices_for_df_modifier_1_sku, default="N/A")

# Use Numpy.where() method to create the required column for sku of 2nd Modifier Option
df_needed_cols["Modifier_2_sku"] = np.where(
    (df_needed_cols["type"].shift(-1) == 'Modifier Option') & (df_needed_cols["type"].shift(-2) == 'Modifier Option'),
    df_needed_cols["sku"].shift(-2), "N/A"
)

# Use Numpy.select() method with lists to create the required column for quantity of 1rst Modifier Option
df_needed_cols["Modifier_1_QTY"] = np.select(conditions_for_df_needed, choices_for_df_needed_QTI_mod_1, default="N/A")

# Use Numpy.select() method with lists to create the required column for quantity of 2nd Modifier Option
df_needed_cols["Modifier_2_QTY"] = np.where(
    (df_needed_cols["type"].shift(-1) == 'Modifier Option') & (df_needed_cols["type"].shift(-2) == 'Modifier Option'),
    df_needed_cols["quantity"].shift(-2), "N/A"
)

In [8]:
# After the creation of additional columns, drop rows with "Modifier Option"
df_needed_cols.drop(df_needed_cols[df_needed_cols["type"] == "Modifier Option"].index, axis=0, inplace=True)

In [9]:
# Drop "type" and "parent_item_sku" columns, trainer names are dropped for confidentiality purposes
df_needed_cols.drop(["type", "parent_item_sku", "Modifier_2_Trainer_Name"], axis=1, inplace=True)

In [10]:
# Reset index
df_needed_cols.reset_index(drop=True, inplace=True)

In [11]:
df_needed_cols.tail()

,order_reference,order_status,status,sku,name,unit_price,quantity,total_price,business_date,Modifier_1_Activity,Modifier_Parent_Item_sku,Modifier_Parent_Item_sku_2,Modifier_1_sku,Modifier_2_sku,Modifier_1_QTY,Modifier_2_QTY
2007,65506,Done,Done,sk-0893,Sports Coaching - Fall Term - 8 sessions,400.0,1,400,2023-10-18,BASKETBALL,sk-0893,N/A,sk-0863,N/A,1.0,N/A
2008,65506,Done,Done,sk-0893,Sports Coaching - Fall Term - 8 sessions,400.0,2,800,2023-10-18,GYMNASTICS,sk-0893,N/A,sk-0860,N/A,2.0,N/A
2009,65518,Done,Done,sk-0923,Stringing - Labor Only,65.0,1,65,2023-10-18,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2010,65574,Done,Done,sk-0885,Fitness PT - STUDENT - per session (valid for ...,80.0,1,80,2023-10-18,FITNESS,sk-0885,sk-0885,sk-0887,sk-0952,1.0,1.0
2011,65580,Done,Done,sk-0902,Bowling single game (with Shoe),20.0,5,100,2023-10-18,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [12]:
# Extract .csv file
df_needed_cols.to_csv("Resulting_File.csv")